In [56]:
# Import necessary libraries
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import os
from pathlib import Path


In [57]:
os.getcwd()

'd:\\TERM 6\\BBM418\\AS1'

In [58]:
def hough_transform(edge_image, rho_resolution=0.5, theta_resolution=0.5, threshold_factor=0.3):
    height, width = edge_image.shape 
    num_theta_bins = int(180 / theta_resolution)
    theta_values = np.arange(0, 180, theta_resolution)
    theta_radians = theta_values * (np.pi / 180)
    diagonal = np.sqrt(height**2 + width**2)
    max_rho = diagonal
    num_rho_bins = int(2 * diagonal / rho_resolution)
    hough_space = np.zeros((num_rho_bins, num_theta_bins))
    cos_theta = np.cos(theta_radians)
    sin_theta = np.sin(theta_radians)
    y, x = np.nonzero(edge_image)
    num_edge_points = len(x)
    x_adjusted = x - (width / 2)
    y_adjusted = y - (height / 2)
    rho = x_adjusted[:, np.newaxis] * cos_theta[np.newaxis, :] + y_adjusted[:, np.newaxis] * sin_theta[np.newaxis, :]
    rho_bins = ((rho + max_rho) / rho_resolution).astype(int)
    valid_mask = (rho_bins >= 0) & (rho_bins < num_rho_bins)
    theta_indices = np.arange(num_theta_bins)[np.newaxis, :]
    theta_indices = np.repeat(theta_indices, num_edge_points, axis=0)
    if num_edge_points > 0:
        np.add.at(hough_space, (rho_bins[valid_mask], theta_indices[valid_mask]), 1)
    max_votes = hough_space.max() if hough_space.max() > 0 else 1
    threshold = 0.8 * max_votes  # for instance
    rho_bins, theta_bins = np.where(hough_space >= threshold)
    votes = hough_space[rho_bins, theta_bins]
    rho = (rho_bins * rho_resolution) - max_rho
    theta = (theta_bins * theta_resolution) * (np.pi / 180)
    peaks = np.column_stack((rho, theta, votes))
    if len(peaks) > 0:
        peaks = peaks[peaks[:, 2].argsort()[::-1]]
    lines = peaks[:, :2] if len(peaks) > 0 else np.empty((0, 2))
    return lines

In [ ]:
for imnum in range(5):
    image = cv2.imread(f'WarpDoc/distorted/curved/{imnum:04d}.jpg', cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Failed to load image {imnum:04d}.jpg")
        continue
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    contrast_image = clahe.apply(image)
    blurred = cv2.GaussianBlur(contrast_image, (9,9), 2)
    resized_image = cv2.resize(blurred, (int(image.shape[1] * 0.25), int(image.shape[0] * 0.25)), interpolation=cv2.INTER_LINEAR)
    edges = cv2.Canny(resized_image, threshold1=100, threshold2=250)
    cv2.imwrite(f'outputs/edges_{imnum:04d}.jpg', edges)
    lines = hough_transform(edges, rho_resolution=1.0, theta_resolution=2.0, threshold_factor=0.5)
    print(f"Image {imnum:04d}: Detected {len(lines)} lines")
    resized_image_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    height, width = resized_image.shape
    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        if b != 0:
            y0 = (rho - 0 * a) / b + height / 2
            y1 = (rho - width * a) / b + height / 2
            pt1 = (0, int(y0))
            pt2 = (width, int(y1))
        else:
            x0 = rho / a + width / 2
            pt1 = (int(x0), 0)
            pt2 = (int(x0), height)
        cv2.line(resized_image_bgr, pt1, pt2, (0, 0, 255), 2)
    cv2.imwrite(f'outputs/lines_{imnum:04d}.jpg', resized_image_bgr)

Image 0000: Detected 1476 lines
Image 0001: Detected 1 lines
Image 0002: Detected 3233 lines
Image 0003: Detected 4556 lines
Image 0004: Detected 3289 lines
